In [1]:
!pip install dfcx-scrapi google-cloud-dialogflow-cx==1.43.0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 29.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a275cdf25ddc4f8f29fd7d6c5a0d09f49084865cbc5c6e33c1ffc026accdf41c
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [1]:
import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

In [2]:
PROJECT_ID = "qwiklabs-gcp-02-a4fd9c600ba4"

In [3]:
from dfcx_scrapi.core.agents import Agents

a = Agents()
agent = a.create_agent(
    project_id=PROJECT_ID,
    display_name="SCRAPI Agents",
    gcp_region="us-west1",
    playbook_agent=True
)

panel = "(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)"
print(f"AGENT LINK: https://vertexaiconversation.cloud.google.com/{agent.name}/{panel}")
print(f"AGENT NAME: {agent.name}")

AGENT LINK: https://vertexaiconversation.cloud.google.com/projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)
AGENT NAME: projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f


In [5]:
import requests
from google.oauth2 import id_token
from google.auth.transport.requests import Request
import google.auth
import json

project_id = "qwiklabs-gcp-02-a4fd9c600ba4"
location_id = "us-west1"

#Replace your Agent Name
agent_name = "projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f"
endpoint_base = "https://us-west1-dialogflow.googleapis.com"

def get_access_token():
    """Gets an access token using google-auth library."""
    creds, project = google.auth.default()
    auth_req = Request()
    creds.refresh(auth_req)
    return creds.token

def update_generative_settings(new_model=None, new_language_code=None):
    """Updates the generative settings for the Conversational agent."""
    access_token = get_access_token()
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    generative_settings_name = f"{agent_name}/generativeSettings"
    url = f"{endpoint_base}/v3/{generative_settings_name}"
    payload = {}
    update_mask_fields = []

    if new_model is not None:
        payload["llmModelSettings"] = {"model": new_model}
        update_mask_fields.append("llmModelSettings.model")

    if new_language_code is not None:
        payload["languageCode"] = new_language_code
        update_mask_fields.append("languageCode")

    if not payload:
        print("No settings to update.")
        return

    params = {"updateMask": ",".join(update_mask_fields)}

    try:
        response = requests.patch(url, headers=headers, params=params, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        updated_settings = response.json()
        print("Generative settings updated successfully:")
        print(json.dumps(updated_settings, indent=2))
        return updated_settings
    except requests.exceptions.RequestException as e:
        print(f"Error updating generative settings: {e}")
        if response is not None:
            print(f"Response status code: {response.status_code}")
            print(f"Response body: {response.text}")
        return None

# Example usage:
update_generative_settings(new_model="gemini-2.5-flash", new_language_code="en")

Generative settings updated successfully:
{
  "languageCode": "en",
  "name": "projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/generativeSettings",
  "llmModelSettings": {
    "model": "gemini-2.5-flash"
  }
}


{'languageCode': 'en',
 'name': 'projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/generativeSettings',
 'llmModelSettings': {'model': 'gemini-2.5-flash'}}

In [6]:
from dfcx_scrapi.core.sessions import Sessions
s = Sessions()
session_id = s.build_session_id(agent.name)
session_id

'projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/sessions/1d9c4040-8aa6-448f-9f6a-c99a150ec2be'

In [7]:
res = s.detect_intent(agent.name, session_id, "hey, how are you?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> hey, how are you?

<font color='green'><b>AGENT RESPONSE:</font></b> I'm doing well, thank you! How can I help you today?

In [8]:
 res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
 s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> I am a large language model, trained by Google.

In [9]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent.name)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbooks_map

{'Default Generative Playbook': 'projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/playbooks/00000000-0000-0000-0000-000000000000'}

In [10]:
playbook = p.get_playbook(playbooks_map["Default Generative Playbook"])
print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: Default goal
INSTRUCTIONS: 


In [11]:
playbook = p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    goal="You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have.",
    instructions=["Greet the user.", "Answer any questions the have to the best of your ability."]
    )

print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

GOAL: You are a friendly Tesla service center agent.
Your job is to help users book appointments and answer any questions they have.
INSTRUCTIONS: steps {
  text: "Greet the user."
}
steps {
  text: "Answer any questions the have to the best of your ability."
}



In [12]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
s.parse_result(res)

<font color='green'><b> USER QUERY:</font></b> what kind of models do you have?

<font color='green'><b>AGENT RESPONSE:</font></b> Hello! I'm a Tesla service center agent. I can help you book appointments and answer your questions.

Regarding your question about what kind of models we have, I need to clarify that I am a service center agent, not a sales agent. My role is to assist with service-related inquiries for existing Tesla vehicles. Therefore, I don't have information on the current models available for purchase. If you're interested in learning about the different Tesla models, I recommend visiting the official Tesla website or contacting a Tesla sales representative.

In [13]:
%mkdir get_weather
%cd get_weather

/content/get_weather


In [14]:
%%writefile requirements.txt
functions-framework==3.*
firebase-admin==6.2.*
firebase-functions
requests==2.*

Writing requirements.txt


In [16]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
!mv get_weather_tool.py main.py
!gcloud config set project $PROJECT_ID
!gcloud functions deploy get_weather_fun --region "us-west1" --runtime python311 --trigger-http --entry-point main --no-allow-unauthenticated --gen2

--2026-01-18 05:05:25--  https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1863 (1.8K) [text/plain]
Saving to: ‘get_weather_tool.py’

get_weather_tool.py 100%[===================>]   1.82K  --.-KB/s    in 0s      

2026-01-18 05:05:25 (23.3 MB/s) - ‘get_weather_tool.py’ saved [1863/1863]

Updated property [core/project].
You can view your function in the Cloud Console here: https://console.cloud.google.com/functions/details/us-west1/get_weather_fun?project=qwiklabs-gcp-02-a4fd9c600ba4

buildConfig:
  automaticUpdatePolicy: {}
  build: projects/1081217401880/locations/us-west1/builds/de4a620b-6dad-4254-9b8a-58b802d56b74
  dockerRegistry: ARTIFACT_REGISTRY
  dockerRep

In [17]:
get_weather_spec = """
openapi: 3.0.0
info:
  title: get_weather
  version: 1.0.0

servers:
  - url: https://get-weather-fun-2zy5tpimiq-uw.a.run.app

paths:
  /get_weather_grid:
    get:
      summary: Returns the current grid information for a city and state
      operationId: get_weather_grid
      parameters:
        - name: latitude
          in: query
          required: true
          schema:
            type: string
        - name: longitude
          in: query
          required: true
          schema:
            type: string
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema:
                type: object
                properties:
                  data:
                    type: string
"""

In [18]:
from dfcx_scrapi.core.tools import Tools

t = Tools()

# Tool 1 - get_weather
weather_tool = t.build_open_api_tool(
    "get_weather_fun",
    spec=get_weather_spec,
    description="Get the current weather for the provided city."
    )
tool1 = t.create_tool(agent.name, weather_tool)
print(str(tool1).lower())


name: "projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/tools/9c7c22a9-ef19-4b2a-8507-2eb60eb3472c"
display_name: "get_weather_fun"
description: "get the current weather for the provided city."
open_api_spec {
  text_schema: "\nopenapi: 3.0.0\ninfo:\n  title: get_weather\n  version: 1.0.0\n\nservers:\n  - url: https://get-weather-fun-2zy5tpimiq-uw.a.run.app\n\npaths:\n  /get_weather_grid:\n    get:\n      summary: returns the current grid information for a city and state\n      operationid: get_weather_grid\n      parameters:\n        - name: latitude\n          in: query\n          required: true\n          schema:\n            type: string\n        - name: longitude\n          in: query\n          required: true\n          schema:\n            type: string\n      responses:\n        \'200\':\n          description: ok\n          content:\n            application/json:\n              schema:\n                type: object\n           

In [19]:
 from dfcx_scrapi.core.playbooks import Playbooks

 p = Playbooks(agent_id=agent.name)

 instructions = [
     "Use the ${TOOL:get_weather_fun} to get the current city/state weather grid information.",
     "Find the latitude and longitude for the location using your own knowledge.",
     "If the user only provides the city you can assume the state unless otherwise specified."]

 # New Playbook
 weather_agent = p.create_playbook(
     agent.name,
     display_name="Weather Agent",
     referenced_tools=[tool1.name],
     goal="You are a senior weather advisor at a network broadcast station. Your job is to predict the weather!",
     instructions=instructions
 )

 playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
 playbooks_map

{'Default Generative Playbook': 'projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/playbooks/00000000-0000-0000-0000-000000000000',
 'Weather Agent': 'projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/playbooks/86869b2c-2568-481d-827c-5a295534bd36'}

In [20]:
p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    instructions=["If the user needs help with the weather, call ${PLAYBOOK: Weather Agent}"]
)

name: "projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/playbooks/00000000-0000-0000-0000-000000000000"
display_name: "Default Generative Playbook"
goal: "You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have."
token_count: 96
referenced_playbooks: "projects/qwiklabs-gcp-02-a4fd9c600ba4/locations/us-west1/agents/ed8c3af9-1033-4983-81c2-6ab456f5255f/playbooks/86869b2c-2568-481d-827c-5a295534bd36"
instruction {
  steps {
    text: "Greet the user."
  }
  steps {
    text: "Answer any questions the have to the best of your ability."
  }
  steps {
    text: "If the user needs help with the weather, call ${PLAYBOOK:Weather Agent}"
  }
}

In [21]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what is the weather like in Austin, Texas?")
s.parse_result(res)


<font color='green'><b> USER QUERY:</font></b> what is the weather like in Austin, Texas?

<font color='green'><b>AGENT RESPONSE:</font></b> Hello! I'm a Tesla service center agent, and I can help you with booking appointments and answering your questions.

Regarding the weather in Austin, Texas, I can connect you with our weather advisor.


<font color='dark red'>TOOL CALL:</font></b> get_weather_grid -> 30.2672

<font color='yellow'>TOOL RESULT:</font></b> {'temperatureUnit': 'F', 'temperature': 27.0}

<font color='green'><b>AGENT RESPONSE:</font></b> The current temperature in Austin, Texas is 27 degrees Fahrenheit.

In [22]:
session_id = s.build_session_id(agent.name)

conversation = [
    "What colors does the model 3 come in?",
    "What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership."
    ]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1


--- TURN 1 --- 



<font color='green'><b> USER QUERY:</font></b> What colors does the model 3 come in?

<font color='green'><b>AGENT RESPONSE:</font></b> Hello! I'm happy to help you with your Tesla questions.

The Tesla Model 3 currently comes in the following colors:

*   Pearl White Multi-Coat
*   Solid Black
*   Midnight Silver Metallic
*   Deep Blue Metallic
*   Red Multi-Coat

Is there anything else I can assist you with today?


--- TURN 2 --- 



<font color='green'><b> USER QUERY:</font></b> What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership.

<font color='dark red'>TOOL CALL:</font></b> get_weather_grid -> 42.3601

<font color='yellow'>TOOL RESULT:</font></b> {'temperatureUnit': 'F', 'temperature': 31.0}

<font color='green'><b>AGENT RESPONSE:</font></b> The current temperature in Boston is 31 degrees Fahrenheit.